In [1]:
import os
import shutil
import re
import json
from natsort import natsorted
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

from config import load_config_data

AZURE_DI_ENDPOINT = load_config_data.get("FORM_RECOGNIZER_ENDPOINT")
AZURE_DI_KEY = load_config_data.get("FORM_RECOGNIZER_API_KEY")
DATA_DIRECTORY = "data"

In [2]:
with open("superset_unique_values.json", 'r') as file:
    data = json.load(file)
uniq_identifier = set(data.get("keys", []))
UNIQUE_IDENTIFIER_KEYS = list(uniq_identifier)
UNIQUE_IDENTIFIER_KEYS.sort()

In [3]:
document_analysis_client = DocumentAnalysisClient(endpoint=AZURE_DI_ENDPOINT, credential=AzureKeyCredential(AZURE_DI_KEY))

In [4]:
def analyze_document(image_path:str):
    """
    Analyzes a single image using Azure Document Intelligence.
    
    Args:
        image_path (str): The full path to the image file.

    Returns:
        A azure ai formrecognizer AnalyzeResult, or None if an error occurs.
    """
    try:
        with open(image_path, "rb") as f:
            poller = document_analysis_client.begin_analyze_document("prebuilt-document", document=f)
            result = poller.result()
            return result
    except Exception as e:
        print(f"      [ERROR] Could not analyze {os.path.basename(image_path)}. Reason: {e}")
        return None

In [5]:
def text_cleaning(text:str) -> str:
    """Perform some basics text cleaning process 

    Args:
        text (str): Input text for the cleaning.

    Returns:
        str: Returned cleaned text
    """
    cleaned_text = re.sub(r'[\\/*?:"<>|]', "", text).strip().replace(" ", "").replace("ZC", "")
    return re.sub(r'-(\d{5})$', '', cleaned_text)

In [6]:
def find_grouping_id(result, identifier_keys):
    """
    Searches the DI result for a unique identifier key based on a priority list
    and returns its value. It checks key-value pairs first, then tables.
    Args:
        result: The result object from Azure DI analysis.
        identifier_keys (list): A prioritized list of keys to search for.
    Returns:
        A unique string "key_value" to be used for grouping, or None if not found.
    """
    for key_to_find in identifier_keys:
        # 1. Search in Key-Value Pairs
        for kv_pair in result.key_value_pairs:
            key_content = kv_pair.key.content.lower().strip().replace(":", "")
            if key_content == key_to_find and kv_pair.value:
                value_content = text_cleaning(kv_pair.value.content)
                print(f"      - Found priority identifier in Key-Value pair -> Key: '{key_to_find}', Value: '{value_content}'")
                # return f"{key_to_find} {text_cleaning(value_content)}"
                return f"{text_cleaning(value_content)}"
            
        # 2. Search in Tables (simple heuristic)
        for table in result.tables:
            for cell in table.cells:
                cell_content = cell.content.lower().strip()
                if cell_content == key_to_find:
                    for other_cell in table.cells:
                        if other_cell.row_index == cell.row_index + 1 and other_cell.column_index == cell.column_index:
                            value_content = text_cleaning(other_cell.content)
                            print(f"      - Found priority identifier in Table -> Key: '{key_to_find}', Value: '{value_content}'")
                            # return f"{key_to_find} {text_cleaning(value_content)}"
                            return f"{text_cleaning(value_content)}"
    # If no prioritized keys were found after checking everything
    return None

# def find_grouping_id(result, identifier_keys):
#     """
#     Searches the DI result for a unique identifier, prioritizing Key-Value Pairs
#     over Tables. Within each source, it respects the priority of identifier_keys.

#     Args:
#         result: The result object from Azure DI analysis.
#         identifier_keys (list): A prioritized list of keys to search for.

#     Returns:
#         A tuple (key, value) if an identifier is found, otherwise None.
#     """
#     found_identifiers = set()
#     # 1. Search in Key-Value Pairs FIRST (High Priority Source)
#     for key_to_find in identifier_keys:
#         for kv_pair in result.key_value_pairs:
#             # Using 'in' for flexibility, e.g., "Ticket No" contains "ticket"
#             # For exact match, use: kv_pair.key.content.lower().strip() == key_to_find
#             if key_to_find == kv_pair.key.content.lower().strip().replace(":", "") and kv_pair.value:
#                 value_content = text_cleaning(kv_pair.value.content)
#                 print(f"      - Found identifier in Key-Value pair -> Key: '{key_to_find}', Value: '{value_content}'")
#                 # return (key_to_find, value_content)
#                 found_identifiers.add((key_to_find, value_content))

#     # 2. If NOTHING was found in KVP, search in Tables (Low Priority Source)
#     for key_to_find in identifier_keys:
#         for table in result.tables:
#             for cell in table.cells:
#                 if key_to_find in cell.content.lower().strip():
#                     # Find the cell in the same row, next column
#                     for other_cell in table.cells:
#                         if other_cell.row_index == cell.row_index + 1 and other_cell.column_index == cell.column_index:
#                             value_content = text_cleaning(other_cell.content)
#                             print(f"      - Found identifier in Table -> Key: '{key_to_find}', Value: '{value_content}'")
#                             # return (key_to_find, value_content)
#                             found_identifiers.add((key_to_find, value_content))
    
#     # If no prioritized keys were found in either source
#     list_identifier = list(found_identifiers)
#     return list_identifier

def find_all_identifiers_on_page(result, identifier_keys):
    """
    Searches the DI result for all unique identifiers with a strict priority:
    Key-Value Pairs are checked first. If any are found, the function returns
    immediately and does not check Tables.

    Args:
        result: The result object from Azure DI analysis.
        identifier_keys (list): A list of keys to search for.

    Returns:
        A list of tuples [(key, value), ...] from EITHER KVP or Tables,
        or an empty list if none are found.
    """

    # 1. Search ONLY in Key-Value Pairs first.
    kvp_identifiers = set()
    for kv_pair in result.key_value_pairs:
        key_content = kv_pair.key.content.lower().strip().replace(":", "")
        for key_to_find in identifier_keys:
            if key_to_find in key_content and kv_pair.value:
                value_content = text_cleaning(kv_pair.value.content)
                kvp_identifiers.add((key_to_find, value_content))

    # --- THIS IS THE CRUCIAL NEW LOGIC ---
    # If we found any identifiers in KVP, we are done. Return them immediately.
    if kvp_identifiers:
        print(f"      - Found identifiers in Key-Value Pairs: {list(kvp_identifiers)}. Prioritizing these and ignoring tables.")
        return list(kvp_identifiers)

    # 2. If and ONLY IF KVP search was empty, proceed to check Tables.
    print("      - No identifiers found in Key-Value Pairs. Checking Tables...")
    table_identifiers = set()
    for table in result.tables:
        for cell in table.cells:
            cell_content = cell.content.lower().strip()
            for key_to_find in identifier_keys:
                if key_to_find in cell_content:
                    # Find the cell in the same row, next column
                    for other_cell in table.cells:
                        if other_cell.row_index == cell.row_index+1 and other_cell.column_index == cell.column_index:
                            value_content = text_cleaning(other_cell.content)
                            table_identifiers.add((key_to_find, value_content))
                            break # Assume first value to the right is the one
    
    if table_identifiers:
        print(f"      - Found identifiers in Tables: {list(table_identifiers)}.")
    
    return list(table_identifiers)

In [ ]:
file_path = r"C:\Users\laxmidhar.routa\Downloads\data_1st_top_50\NORTH TEXAS TOLLWAY AUTHORITY\2024096-0000199-001-01.jpg"
result = analyze_document(file_path)
grouping_ids = find_grouping_id(result, UNIQUE_IDENTIFIER_KEYS)

In [ ]:
grouping_ids

In [7]:
def organize_files(authority_path, image_groups):
    """
    Creates notice folders using the provided keys and moves the grouped images into them.

    Args:
        authority_path (str): The path to the current authority folder.
        image_groups (dict): A dictionary mapping final folder names (e.g., 'notice_1') to lists of image paths.
    """
    print("\n  Organizing files into notice folders...")
    
    for notice_folder_name, image_paths in image_groups.items():
        notice_folder_path = os.path.join(authority_path, notice_folder_name)
        os.makedirs(notice_folder_path, exist_ok=True)
        print(f"    - Creating folder: {notice_folder_name}")
        for image_path in image_paths:
            file_name = os.path.basename(image_path)
            destination_path = os.path.join(notice_folder_path, file_name)
            print(f"      - Moving {file_name} to {notice_folder_name}")
            shutil.move(image_path, destination_path)

In [11]:
def is_page_blank(result, threshold=0.6):
    """
    Checks if the analyzed document result is effectively blank.

    Args:
        result: The result object from Azure DI analysis.
        threshold (float): Minimum average word confidence required to consider a page blank.

    Returns:
        True if the page is blank, False otherwise.
    """
    if not result.pages:
        return True

    words = [word.confidence for page in result.pages for word in page.words]

    if not words:
        return True

    avg_confidence = sum(words) / len(words)
    # print(f"{avg_confidence:.4f}")
    return avg_confidence < threshold

In [9]:
def main(authority_name):
    """
    Main function to orchestrate the notice grouping process.
    """
    print("--- Starting Notice Grouping ---")

    # Iterate through each authority folder in the data directory
    # for authority_name in os.listdir(DATA_DIRECTORY):
    # DATA_DIRECTORY = r"C:\Users\laxmidhar.routa\Downloads\30 IA records"
    authority_path = os.path.join(DATA_DIRECTORY, authority_name)
    
    print(f"\nProcessing Authority: {authority_name}")
    
    image_groups = {}
    unprocessed_images = []
    last_found_grouping_id = None 
    current_group_key = None      
    notice_counter = 0
        

    image_files = [f for f in os.listdir(authority_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    sorted_files = natsorted(image_files)
        
    for image_name in sorted_files:
        image_path = os.path.join(authority_path, image_name)
        
        analysis_result = analyze_document(image_path)

        if is_page_blank(analysis_result):
            print(f"      - Detected blank page: {image_name}. Adding to unprocessed list.")
            unprocessed_images.append(os.path.basename(image_path))
            continue

        current_image_id = find_grouping_id(analysis_result, UNIQUE_IDENTIFIER_KEYS)

        if current_image_id and current_image_id != last_found_grouping_id:
            last_found_grouping_id = current_image_id
            notice_counter += 1
            current_group_key = f"notice_{notice_counter}"
            print(f"      - New notice detected. Assigning to: {current_group_key} (value: {last_found_grouping_id})")

        if current_group_key:
                if not current_image_id:
                    print(f"      - No identifier found. Associating with previous notice: {current_group_key}")
                if current_group_key not in image_groups:
                    image_groups[current_group_key] = []
                # image_groups[current_group_key].append(image_path)
                image_groups[current_group_key].append(os.path.basename(image_path))
        else:
                print("      - No identifier found and no previous notice to associate with.")
                unprocessed_images.append(image_path)
    return image_groups, unprocessed_images

In [ ]:
authority = ["OKLAHOMA TURNPIKE AUTHORITY PLATEPAY", "Illinois Tollway", "FDOT", "NORTH TEXAS TOLLWAY AUTHORITY", "TOLLS BY MAIL MTABT"]
for auth in authority:
    main(auth)

In [13]:
def main_1(authority_name):
    """
    Main function to orchestrate the notice grouping process.
    """
    # ... (Your credential validation and client initialization code remains the same) ...
    # Validate credentials
    
    print("--- Starting Notice Grouping ---")

    # for authority_name in os.listdir(DATA_DIRECTORY):
    # DATA_DIRECTORY = r"C:\Users\laxmidhar.routa\Downloads\30 IA records"
    authority_path = os.path.join(DATA_DIRECTORY, authority_name)
    
    print(f"\nProcessing Authority: {authority_name}")
    
    image_groups = {}
    unprocessed_images = []
    
    current_notice_identifier = None 
    current_group_name = None      
    notice_counter = 0

    image_files = [f for f in os.listdir(authority_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    sorted_images = natsorted(image_files)
    
    for image_name in sorted_images:
        image_path = os.path.join(authority_path, image_name)
        
        analysis_result = analyze_document(image_path)

        if is_page_blank(analysis_result):
            print(f"      - Detected blank page: {image_name}. Adding to unprocessed list.")
            unprocessed_images.append(image_path)
            continue

        all_ids_on_page = find_all_identifiers_on_page(analysis_result, UNIQUE_IDENTIFIER_KEYS)

        is_new_notice = False
        if current_notice_identifier and current_notice_identifier in all_ids_on_page:
            print(f"      - Continuing with '{current_group_name}' because tracked ID {current_notice_identifier} was found.")
            is_new_notice = False
        elif all_ids_on_page:
            is_new_notice = True
        
        if is_new_notice:
            highest_priority_id = None
            for key in UNIQUE_IDENTIFIER_KEYS:
                for found_id in all_ids_on_page:
                    if found_id[0] == key:
                        highest_priority_id = found_id
                        break
                if highest_priority_id:
                    break
            
            current_notice_identifier = highest_priority_id
            notice_counter += 1
            current_group_name = f"notice_{notice_counter}"
            key, value = current_notice_identifier
            print(f"      - New notice detected. Assigning to folder: '{current_group_name}' (Now tracking by: ('{key}', '{value}'))")

        if current_group_name:
            if current_group_name not in image_groups:
                image_groups[current_group_name] = []
            image_groups[current_group_name].append(os.path.basename(image_path))
        else:
            unprocessed_images.append(os.path.basename(image_path))

    if image_groups:
        # organize_files(authority_path, image_groups)
        print(image_groups)
    
    if unprocessed_images:
        print("\n  The following images could not be grouped (or were blank):")
        for img_path in unprocessed_images:
            print(f"    - {os.path.basename(img_path)}")


In [14]:
authority = ["OKLAHOMA TURNPIKE AUTHORITY PLATEPAY", "Illinois Tollway", "FDOT", "NORTH TEXAS TOLLWAY AUTHORITY", "TOLLS BY MAIL MTABT"]
for auth in authority:
    print(main_1(auth))

--- Starting Notice Grouping ---

Processing Authority: OKLAHOMA TURNPIKE AUTHORITY PLATEPAY
      - Found identifiers in Key-Value Pairs: [('violation', '0015932311-2'), ('notice number', '0015932311-2')]. Prioritizing these and ignoring tables.
      - New notice detected. Assigning to folder: 'notice_1' (Now tracking by: ('notice number', '0015932311-2'))
      - No identifiers found in Key-Value Pairs. Checking Tables...
{'notice_1': ['2024095-0002505-001-01.jpg', '2024095-0002505-001-02.jpg']}
None
--- Starting Notice Grouping ---

Processing Authority: Illinois Tollway
      - Found identifiers in Key-Value Pairs: [('invoice number', 'VN5306104704')]. Prioritizing these and ignoring tables.
      - New notice detected. Assigning to folder: 'notice_1' (Now tracking by: ('invoice number', 'VN5306104704'))
      - No identifiers found in Key-Value Pairs. Checking Tables...
      - Found identifiers in Key-Value Pairs: [('invoice number', 'VN5106273430')]. Prioritizing these and igno

In [ ]:
main_1("FORT BEND COUNTY TOLL ROAD AUTHORITY") #"CITY OF CHICAGO"  

In [ ]:
import fitz
from PIL import Image
import os

In [ ]:
def pdf_text_bbox_image(page):
    '''This method takes a page of a PDF document as a input and returns list of words and the bounding box of the words and image of the page '''
    # text = [data[4] for data in page.get_text_words()]
    # bbox = [data[0:4] for data in page.get_text_words()]
    pix = page.get_pixmap(matrix = fitz.Matrix(2, 2))
    img = Image.frombytes('RGB', [pix.width, pix.height], pix.samples)
    # bbox_arry = np.array(bbox, dtype=np.int64)
    # return text, bbox_arry, img
    return img

In [ ]:
file_path = r"C:\Users\laxmidhar.routa\Downloads\New Jersey Turnpike Authority\Notice notice 1.pdf"
file_name = os.path.basename(file_path).split(".")[0]
output_dir = os.path.abspath(file_path)
with fitz.open(file_path) as p_file:
    for pos, page in enumerate(p_file):
        img = pdf_text_bbox_image(page)
        img.save(f"{file_name}-Page-{pos+1}.png")

Updated Logic for running it in local


In [ ]:
import json
import os
import shutil
import re
from natsort import natsorted
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import logging

from config import load_config_data
AZURE_DI_ENDPOINT = load_config_data["FORM_RECOGNIZER_ENDPOINT"]
AZURE_DI_KEY = load_config_data["FORM_RECOGNIZER_API_KEY"]
logger = logging.getLogger('custom')

In [ ]:
with open("superset_unique_values.json", 'r') as file:
    data = json.load(file)
unique_identifier_raw = set(data.get("keys", []))
UNIQUE_IDENTIFIER_KEYS = [item.lower().strip().replace(":", "") for item in unique_identifier_raw]
UNIQUE_IDENTIFIER_KEYS.sort()

In [ ]:
form_recognizer_client = DocumentAnalysisClient(endpoint=AZURE_DI_ENDPOINT, credential=AzureKeyCredential(AZURE_DI_KEY))

In [ ]:
def text_cleaning(text: str) -> str:
    cleaned = re.sub(r'[\\/*?:"<>|]', "", text).strip().replace(" ", "")
    return re.sub(r'-(\d{5})$', '', cleaned)

In [ ]:
def find_grouping_id(result, identifier_keys):
    """
    Searches the DI result for a unique identifier key based on a priority list
    and returns its value. It checks key-value pairs first, then tables.
    Args:
        result: The result object from Azure DI analysis.
        identifier_keys (list): A prioritized list of keys to search for.
    Returns:
        A unique string "key_value" to be used for grouping, or None if not found.
    """
    for key_to_find in identifier_keys:
        # 1. Search in Key-Value Pairs
        for kv_pair in result.get("key_value_pairs", []):
            key_content = kv_pair.get("key").get("content").lower().strip().replace(":", "")
            if key_content == key_to_find and kv_pair.get("value"):
                value_content = text_cleaning(kv_pair.get("value").get("content"))
                print(f"      - Found priority identifier in Key-Value pair -> Key: '{key_to_find}', Value: '{value_content}'")
                # return f"{key_to_find} {text_cleaning(value_content)}"
                return f"{text_cleaning(value_content)}"
            
        # 2. Search in Tables (simple heuristic)
        for table in result.get("tables"):
            for cell in table.get("cells"):
                cell_content = cell.get("content").lower().strip()
                if cell_content == key_to_find:
                    for other_cell in table.get("cells"):
                        if other_cell.get("row_index") == cell.get("row_index") + 1 and other_cell.get("column_index") == cell.get("column_index"):
                            value_content = text_cleaning(other_cell.get("content"))
                            print(f"      - Found priority identifier in Table -> Key: '{key_to_find}', Value: '{value_content}'")
                            # return f"{key_to_find} {text_cleaning(value_content)}"
                            return f"{text_cleaning(value_content)}"
    # If no prioritized keys were found after checking everything
    return None


In [ ]:
def group_pages_by_identifier(pages_data, identifier_keys):
    logger.info("Started grouping pages using identifier keys.")
    
    image_groups = {}
    unprocessed_images = []
    last_found_grouping_id = None 
    current_group_key = None      
    notice_counter = 0

    for page_number in natsorted(pages_data.keys()):
        di_data = pages_data[page_number]
        # di_data = page.get("original_di_data", {})

        grouping_id = find_grouping_id(di_data, identifier_keys)

        if grouping_id and grouping_id != last_found_grouping_id:
            last_found_grouping_id = grouping_id
            notice_counter += 1
            current_group_key = f"notice_{notice_counter}"
            logger.info(f"      - New notice detected. Assigning to: {current_group_key} (value: {last_found_grouping_id})")

        if current_group_key:
            if not grouping_id:
                logger.info(f"      - No identifier found. Associating with previous notice: {current_group_key}")
                if current_group_key not in image_groups:
                    image_groups[current_group_key] = []
                # image_groups[current_group_key].append(image_path)
                image_groups[current_group_key].append(os.path.basename(page_number))
        else:
            logger.info("      - No identifier found and no previous notice to associate with.")
            unprocessed_images.append(page_number)
    return image_groups, unprocessed_images

In [ ]:
folder_path = r"C:\Users\laxmidhar.routa\Downloads\data_1st_top_50\NORTH TEXAS TOLLWAY AUTHORITY"
di_page_data = {}
for item in os.listdir(folder_path):
    image_path = os.path.join(folder_path, item)
    with open(image_path, "rb") as f:
        poller = form_recognizer_client.begin_analyze_document("prebuilt-document", document=f)
        result = poller.result()
        di_page_data[os.path.basename(image_path)] = result.to_dict()

In [ ]:
group_pages_by_identifier(di_page_data, UNIQUE_IDENTIFIER_KEYS)

In [ ]:
main_dir_path = r"C:\Users\laxmidhar.routa\Downloads\Project Work\data"
output_dir_path = r"C:\Users\laxmidhar.routa\Desktop\Test documents"
folder_names = []

In [ ]:
import shutil, os

for item in folder_names:
    src_path = os.path.join(main_dir_path, item)
    dst_path = os.path.join(output_dir_path, item)
    shutil.copytree(src_path, dst_path)